Get bathymetry/backscatter data without downloading data locally to computer

In [4]:
import shutil, os, html, requests

In [5]:
def download_file(url):
    local_filename = os.path.join('data',url.split('/')[-1])
    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename

For bathymetry data (25m):

In [ ]:
download_file('https://gsi.geodata.gov.ie/downloads/Marine/Data/Downloads/LatestEntireAreaMerge/IE_GSI_MI_Bathymetry_25m_IE_Waters_WGS84_LAT_TIFF.zip')

For backscatter data (40m):

In [ ]:
download_file('https://gsi.geodata.gov.ie/downloads/Marine/Data/Downloads/LatestEntireAreaMerge/IE_GSI_MI_Backscatter_40m_Offshore_IE_WGS84_LAT_TIFF.zip')